## q<sup>2</sup> Tutorial - 3. Working with a sample of stars

Let's say that we have N stars and we want to derive spectroscopic parameters for all of them. We could use q2.specpars.solve_one on each star, as we did for Arcturus in Part 2 of this Tutorial, but q<sup>2</sup> offers a better option: q2.specpars.solve_all. As the name suggests, this one will solve for parameters for all stars in our input files.

For this part of the Tutorial we will use data for a couple of Hyades stars. Take a look at the "hyades_stars.csv" and "hyades_lines.csv" files to know what we are working with. Note that the files contain a star with id=vestaOct, which corresponds to a solar spectrum to be used as reference.

*In Parts 1 and 2 of this Tutorial, both stars had EW measurements for all lines, but q<sup>2</sup> can identify missing data and will have no problems performing calculations using incomplete linelists. The Hyades lines file that we are employing here has some missing EW data, on purpose, to highlight this feature.*

As usual, we begin by importing q<sup>2</sup> and uploading the data:

In [1]:
import q2
data = q2.Data('hyades_stars.csv', 'hyades_lines.csv')

The q2.specpars.solve_all function works very much like solve_one, except that instead of applying it to a star object, we apply it to the entire data object, as shown below. q<sup>2</sup> will figure out which stars need to be solved (excludes the reference, for example, which in this case is vestaOct --we are doing differential analysis-- and stars that do not have any EWs in the lines csv file --that is not the case of this example, though).

Note that in this example we are not setting the steps for SolvePars (step_teff, etc.), so they will be large at the beginning (however, it is required that a SolvePars object is sent to the solve_all function, with necessary parameters such as the model atmosphere grid to use). As the parameters converge within those steps, they will reduce automatically. This is common practice when we work on a sample of stars with unknown parameters. Since we do know that these objects are solar-type stars, we have set the input parameters for all of them equal to solar in the csv file.

When working on a sample of stars, the results are saved to an output csv file, which we are calling "hyades_solution.csv". The output on screen can be informative as well, so we should keep an eye on it (if the number of stars is large, it may be best to send it to a logfile):

In [2]:
sp = q2.specpars.SolvePars()
sp.grid = 'marcs'

q2.specpars.solve_all(data, sp, 'hyades_solution.csv', 'vestaOct')

------------------------------------------------------
Initializing ...
- Date and time: 15-Jan-2016, 13:34:41
- Model atmospheres: marcs
- Star data: hyades_stars.csv
- Line list: hyades_lines.csv
------------------------------------------------------

*******
hd28099
*******
it Teff logg [Fe/H]  vt           [Fe/H]
-- ---- ---- ------ ----      --------------
 0 5777 4.44  0.000 1.00 --->  0.171+/-0.034
 1 5745 4.76  0.171 1.32 --->  0.088+/-0.083
 2 5777 4.44  0.088 1.00 --->  0.180+/-0.034
 3 5745 4.76  0.180 1.32 --->  0.090+/-0.083
 4 5777 4.44  0.090 1.00 --->  0.180+/-0.034
-- Converged at iteration 4 of 50
 5 5761 4.60  0.180 1.16 --->  0.137+/-0.045
 6 5777 4.44  0.137 1.00 --->  0.185+/-0.035
 7 5761 4.60  0.185 1.16 --->  0.138+/-0.045
-- Converged at iteration 7 of 50
 8 5769 4.52  0.138 1.08 --->  0.159+/-0.031
 9 5761 4.44  0.159 1.16 --->  0.143+/-0.030
10 5769 4.36  0.143 1.24 --->  0.130+/-0.032
11 5777 4.44  0.130 1.32 --->  0.115+/-0.032
12 5785 4.36  0.115 1.24 ---

Have a look at the output file (hyades_solution.csv). It contains all the final parameters that we would now use to derive abundances of elements other than iron, for example, or whatever it is that we need them for. Note that there is a column "feh_model" and another column "feh". The former is the [Fe/H] used to interpolate the final model atmosphere while the latter is the [Fe/H] that results from the spectral line analysis. They should be very close to each other, if not identical, if convergence was achieved.

We should always check the column 'converged'. They should all be True; otherwise we need to re-iterate.

Automating MOOG has many advantages. Let's say, for example, that for whatever reason we want to re-do all the calculations done above, but using Kurucz's ODFNEW model atmospheres instead of MARCS. All we have to do then is re-define sp.grid and run q2.specpars.solve_all again. We will do that now.

We will also do another important calculation. By default, q2.specpars.solve_all does not calculate the formal errors (hence the -1 values in the error columns of the output file). The reason is that error calculation can be time-consuming and we only really need to do it once we are happy with our solutions. We will now turn-on error calculation by changing the errors attribute of the SolvePars object to True.

For this next step, we will set the PlotPars object "pp" so that the range of [Fe/H] in the plots is the same for all stars. In addition, the figures will be saved in eps format. By default, q<sup>2</sup> saves figures in png format, but that can be changed via the figure_format attribute of the PlotPars object. One last thing: to keep things in order, we will send the plots to a directory odfnew (the directory itself will be created by q<sup>2</sup> if it doesn't exist).

In [3]:
sp.grid = 'odfnew'
sp.errors = True

pp = q2.specpars.PlotPars()
pp.afe = [-0.05, 0.35]
pp.figure_format = 'eps'
pp.directory = 'odfnew'

q2.specpars.solve_all(data, sp, 'hyades_solution_odfnew_err.csv',
                      'vestaOct', pp)

------------------------------------------------------
Initializing ...
- Date and time: 15-Jan-2016, 13:35:33
- Model atmospheres: odfnew
- Star data: hyades_stars.csv
- Line list: hyades_lines.csv
------------------------------------------------------

*******
hd28099
*******
it Teff logg [Fe/H]  vt           [Fe/H]
-- ---- ---- ------ ----      --------------
 0 5777 4.44  0.000 1.00 --->  0.172+/-0.035
 1 5745 4.76  0.172 1.32 --->  0.093+/-0.083
 2 5777 4.44  0.093 1.00 --->  0.182+/-0.034
 3 5745 4.76  0.182 1.32 --->  0.094+/-0.083
 4 5777 4.44  0.094 1.00 --->  0.182+/-0.034
-- Converged at iteration 4 of 50
 5 5761 4.60  0.182 1.16 --->  0.144+/-0.045
 6 5777 4.44  0.144 1.00 --->  0.189+/-0.035
 7 5761 4.60  0.189 1.16 --->  0.145+/-0.045
-- Converged at iteration 7 of 50
 8 5769 4.52  0.145 1.08 --->  0.166+/-0.032
 9 5761 4.44  0.166 1.16 --->  0.146+/-0.030
10 5769 4.36  0.146 1.24 --->  0.132+/-0.032
11 5777 4.44  0.132 1.32 --->  0.117+/-0.033
12 5785 4.36  0.117 1.24 --

Let's have a look at the output file hyades\_solution\_odfnew\_err.csv. Our final "spectroscopic" parameters and their formal errors are there (teff, logg, etc.). The information from the last iteration of iron\_stats is also copied there for reference. Since we requested formal errors to be computed, column err\_feh contains that information now. Previously, err\_feh was simply the 1-sigma line-to-line scatter of the iron abundances. Note that both output files have err\_feh and err\_feh\_ columns. The former is the preferred value.

The most tedious part of elemental abundance work is the derivation of fundamental parameters Teff, logg, vt. Once those are set (as they are now in this Hyades example) deriving abundances for elements other than iron is really straightforward. In Part 4 of this Tutorial we tackle that problem.